In [10]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pylab as plt
import pandas as pd

In [11]:
class I:
    U = 0
    FL = 1
    SL = 2
    I = 3
    Tx = 4
    
    
class A:
    Inc = 0
    Mor = 1
    Covered = 2
    Eli = 3
    Yield = 4
    PCF = 5

    
    
class ModelTB:
    def get_y0(self):
        y0 = np.array([[970, 10, 10, 10, 0], np.zeros(5)]).T / 1000
        a0 = np.zeros(6)
        return np.concatenate([y0.reshape(-1), a0])
    
    def intervene(self, t, y, dy, da, acf):
        if acf['Cohort']:
            r_acf = - np.log(1 - acf['Coverage'])
            r_acf = min(r_acf, 20)
        else:
            r_acf = acf['Coverage']
        switch = acf['Switch']
        acf = r_acf * y[:, 0]
        eli = acf * np.array([1, 1, 1, 1, 0])
        
        da[A.Covered] += acf.sum()
        da[A.Eli] += eli.sum()
        da[A.Yield] += eli[I.I]
        
        if switch:
            dy[:, 0] -= eli
            dy[0:3, 1] += eli[0:3]
            dy[4, 1] += eli[3]
            dy[:, 0] += y[:, 1]
            dy[:, 1] -= y[:, 1]
        else:
            dy[3, 0] -= eli[3]
            dy[4, 1] += eli[3]
        
        return dy, da
    
    def __call__(self, t, ya, pars, acf=None):
        y, aux = ya[:-6], ya[-6:]
        y = y.reshape((5, 2))
        
        foi = pars['beta'] * y[I.I].sum()
        lat = pars['r_lat'] * y[I.FL]
        act = pars['r_act'] * y[I.FL]
        react = pars['r_react'] * y[I.SL]

        det = pars['r_det'] * y[I.I]
        txo = pars['r_tx'] * y[I.Tx]

        die_tb = pars['r_die_tb'] * y[I.I]
        r_die_bg = pars['r_die_bg'] - die_tb.sum() / y.sum()
        r_die_bg = max(r_die_bg, 0)
        die_bg = r_die_bg * y
        sc = pars['r_sc'] * y[I.I]

        dy = np.zeros_like(y)
        
        dy[I.U] -= foi * y[I.U]
        dy[I.FL] += foi * (y[I.U] + (1 - pars['p_im']) * y[I.SL]) - act - lat
        dy[I.SL] += lat - foi * (1 - pars['p_im']) * y[I.SL] + txo + sc - react
        dy[I.I] += act + react - det - sc
        dy[I.Tx] += det - txo

        dy -= die_bg
        dy[I.I] -= die_tb
        dy[I.U, 0] += die_bg.sum() + die_tb.sum()

        da = np.zeros_like(aux)
        da[A.Inc] += (act + react).sum()
        da[A.Mor] += die_tb.sum()
        
        if acf is not None:
            dy, da = self.intervene(t, y, dy, da, acf)
        
        return np.concatenate([dy.reshape(-1), da.reshape(-1)])
    
    def mea(self, t, ya):
        aux = ya[-6:]

        return {
            'Time': t,
            'Inc': aux[A.Inc],
            'Mor': aux[A.Mor],
            'Covered': aux[A.Covered],
            'Eli': aux[A.Eli],
            'Yield': aux[A.Yield],
            'PCF': aux[A.PCF]
        }

In [12]:
p = {
    'beta': 15,
    'r_act': 0.5 * 0.1 / 0.9,
    'r_lat': 0.5,
    'r_react': 0.002,
    'r_det': 1,
    'r_tx': 2,
    'r_die_tb': 0.1,
    'r_sc': 0.2,
    'r_die_bg': 0.05,
    'p_im': 0.6
}

In [13]:
model = ModelTB()
y0 = model.get_y0()

sol = solve_ivp(model, t_span = [0, 500], y0 = y0, args=(p, ))
ya0 = sol.y[:, -1]
ya0[-6:] = 0

In [14]:

def exp_discrete(model, ya0, coverage=0.1, interval=0.5):
    ts = np.linspace(0, 10, int(10 / interval) + 1)
    ya = ya0
    mss = list()
    mss.append(model.mea(0, ya))

    for t0, t1 in zip(ts[:-1], ts[1:]):
        sol = solve_ivp(model, t_span = [t0, t1], y0 = ya, args=(p, ))
        ya = sol.y[:, -1]

        ## ACF
        cov = coverage * interval
        y, aux = ya[:-6], ya[-6:]
        y = y.reshape((-1, 2))
        prop =  cov / y[:, 0].sum()
        prop = min(prop, 1)
        screened = y[:, 0] * prop
        eli = screened * np.array([1, 1, 1, 1, 0])
        found = eli[I.I]
        y[:, 0] -= eli
        y[0:3, 1] += eli[0:3]
        y[4, 1] += eli[3]
        aux[A.Covered] += screened.sum()
        aux[A.Eli] += eli.sum()
        aux[A.Yield] += eli[3]
        ya = np.concatenate([y.reshape(-1), aux])

        if round(t1) == t1:
            y, aux = ya[:-6], ya[-6:]
            y = y.reshape((-1, 2))
            y[:, 0] += y[:, 1]
            y[:, 1] = 0
            ya = np.concatenate([y.reshape(-1), aux])
            mss.append(model.mea(t1, ya))

    mss = pd.DataFrame(mss).assign(Coverage = coverage, Interval = interval)
    return mss

In [15]:
acf = {'Coverage': 0.1, 'Switch': True, 'Cohort': False}

ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, acf), dense_output=True)
pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)]).assign(**acf)

,Time,Inc,Mor,Covered,Eli,Yield,PCF,Coverage,Switch,Cohort
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.1,True,False
1,1.0,0.002919,0.000210,0.096475,0.096370,0.000208,0.0,0.1,True,False
2,2.0,0.005789,0.000412,0.189422,0.189216,0.000403,0.0,0.1,True,False
3,3.0,0.008607,0.000610,0.281253,0.280949,0.000591,0.0,0.1,True,False
4,4.0,0.011378,0.000804,0.372729,0.372330,0.000775,0.0,0.1,True,False
5,5.0,0.014108,0.000995,0.464092,0.463599,0.000955,0.0,0.1,True,False
6,6.0,0.016803,0.001183,0.555421,0.554836,0.001133,0.0,0.1,True,False
7,7.0,0.019467,0.001369,0.646738,0.646062,0.001309,0.0,0.1,True,False
8,8.0,0.022105,0.001553,0.738051,0.737286,0.001483,0.0,0.1,True,False
9,9.0,0.024718,0.001735,0.829363,0.828510,0.001655,0.0,0.1,True,False


In [16]:
acf = {'Coverage': 0.2, 'Switch': False, 'Cohort': True}

ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, acf), dense_output=True)
pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)]).assign(**acf)

,Time,Inc,Mor,Covered,Eli,Yield,PCF,Coverage,Switch,Cohort
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.2,False,True
1,1.0,0.002903,0.000201,0.223093,0.222865,0.000449,0.0,0.2,False,True
2,2.0,0.005699,0.000386,0.446096,0.445658,0.000862,0.0,0.2,False,True
3,3.0,0.008384,0.000562,0.669018,0.668384,0.001253,0.0,0.2,False,True
4,4.0,0.010971,0.000731,0.891867,0.891046,0.001629,0.0,0.2,False,True
5,5.0,0.013476,0.000893,1.114650,1.113649,0.001992,0.0,0.2,False,True
6,6.0,0.015910,0.001051,1.337374,1.336199,0.002343,0.0,0.2,False,True
7,7.0,0.018283,0.001204,1.560042,1.558698,0.002685,0.0,0.2,False,True
8,8.0,0.020603,0.001354,1.782660,1.781151,0.003018,0.0,0.2,False,True
9,9.0,0.022878,0.001501,2.005232,2.003563,0.003344,0.0,0.2,False,True


In [17]:
ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, {'Coverage': 0.5, 'Switch': False, 'Cohort': False}), dense_output=True)
pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)])

,Time,Inc,Mor,Covered,Eli,Yield,PCF
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1.0,0.002868,0.000184,0.499763,0.499272,0.000920,0.0
2,2.0,0.005523,0.000338,0.999134,0.998236,0.001688,0.0
3,3.0,0.007970,0.000476,1.498180,1.496925,0.002381,0.0
4,4.0,0.010246,0.000605,1.996948,1.995368,0.003021,0.0
5,5.0,0.012382,0.000724,2.495475,2.493594,0.003617,0.0
6,6.0,0.014404,0.000837,2.993792,2.991629,0.004179,0.0
7,7.0,0.016332,0.000944,3.491925,3.489496,0.004712,0.0
8,8.0,0.018181,0.001047,3.989895,3.987210,0.005220,0.0
9,9.0,0.019962,0.001145,4.487721,4.484794,0.005709,0.0


In [18]:
ys = solve_ivp(model, t_span = [0, 10], y0 = ya0, args=(p, {'Coverage': 0.5, 'Switch': False, 'Cohort': True}), dense_output=True)
pd.DataFrame([model.mea(t, ys.sol(t)) for t in np.linspace(0, 10, 11)])

,Time,Inc,Mor,Covered,Eli,Yield,PCF
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1.0,0.002847,0.000173,0.692712,0.692049,0.001202,0.0
2,2.0,0.005418,0.000310,1.384738,1.383561,0.002145,0.0
3,3.0,0.007734,0.000429,2.076223,2.074612,0.002974,0.0
4,4.0,0.009846,0.000538,2.767257,2.765261,0.003723,0.0
5,5.0,0.011795,0.000637,3.457910,3.455565,0.004410,0.0
6,6.0,0.013616,0.000730,4.148238,4.145570,0.005045,0.0
7,7.0,0.015332,0.000816,4.838288,4.835319,0.005642,0.0
8,8.0,0.016963,0.000899,5.528098,5.524846,0.006206,0.0
9,9.0,0.018521,0.000977,6.217698,6.214179,0.006743,0.0


In [19]:
res = list()

for c in [0.1, 0.2, 0.5, 1]:
    for i in [1, 0.5, 0.25, 0.1, 0.05, 0.1]:
        res.append(exp_discrete(model, ya0, coverage=c, interval=i))

res = pd.concat(res)
res

,Time,Inc,Mor,Covered,Eli,Yield,PCF,Coverage,Interval
0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.1,1.0
1,1.0,0.002934,0.000217,0.1,0.099894,0.000217,0.0,0.1,1.0
2,2.0,0.005829,0.000422,0.2,0.199789,0.000427,0.0,0.1,1.0
3,3.0,0.008667,0.000620,0.3,0.299688,0.000631,0.0,0.1,1.0
4,4.0,0.011451,0.000814,0.4,0.399588,0.000831,0.0,0.1,1.0
...,...,...,...,...,...,...,...,...,...
6,6.0,0.011791,0.000490,6.0,5.991950,0.006916,0.0,1.0,0.1
7,7.0,0.013062,0.000538,7.0,6.991153,0.007583,0.0,1.0,0.1
8,8.0,0.014247,0.000581,8.0,7.990421,0.008201,0.0,1.0,0.1
9,9.0,0.015361,0.000623,9.0,8.989740,0.008782,0.0,1.0,0.1


In [18]:
r0 = res[res.Coverage == 0.5]
r0[r0.Time == 10]

,Time,Inc,Mor,Covered,Eli,Yield,PCF,Coverage,Interval
10,10.0,0.021284,0.001188,5.0,4.996394,0.006918,0.0,0.5,1.00
10,10.0,0.021099,0.001169,5.0,4.996007,0.006812,0.0,0.5,0.50
10,10.0,0.020972,0.001156,5.0,4.995721,0.006784,0.0,0.5,0.25
10,10.0,0.020883,0.001147,5.0,4.995515,0.006776,0.0,0.5,0.10
10,10.0,0.020851,0.001144,5.0,4.995440,0.006775,0.0,0.5,0.05
10,10.0,0.020883,0.001147,5.0,4.995515,0.006776,0.0,0.5,0.10


In [20]:
res.to_csv('out/test.csv')